# RFID tag reader with Arduino
This is a sample notebook containing some tips to read RFID tags with Arduino and controlling this remotely in Python. 
Our strategy is going to be based on the following: 

- We are going to use Python to manage all data and control the Arduino board. First we will use dictionaries to store the data and later we will update this to a database.
- We will use the Arduino board to read the RFID tags and send the data to Python. We will only send the data when a new tag is read. 
- The Arduino board will be connected to the computer via USB.
- We will use the serial port to communicate between Python and Arduino.
- The Python code will basically do the following: 
- Send a message to the Arduino board to verify the connection. We will call this message "handshake".
- Read the serial port and check if there is new data.
- When there is new data, we will check if it is a "handshake" message or a tag ID.
- If it is a tag ID, we will check if it is already in the database. If it is not, we will add it to the database. If it is, we will update the last time it was read.
- We will check if the tag is assign to a person. If it is, we will send a message back to Arduino with the message. The Arduino will show this message in the LCD screen.

## Python code
Find below a skeleton of the Python code. We will use the pyserial library to communicate with the Arduino board.

```python
import serial
import time

# This is a dictionary with the users and their tag IDs. We can add more fields as needed, to build additional functionality (for instance to distinguish between students and teachers)
users = [{
    "tagId": "1234567890",
    "name": "John Doe"
}, {
    "tagId": "0987654321",
    "name": "Jane Doe"}]

# These are the tags that have been read. We will use this to check if the tag has been read before
tags = [{
    "tagId": "1234567890",
    "lastRead": "2020-01-01 00:00:00"
}, {
    "tagId": "0987654321",
    "lastRead": "2020-01-01 00:00:00"}]

# This is the serial port. We will use this to communicate with the Arduino board
port = 'COM5'

arduino = serial.Serial(port, 9600, timeout=1)
time.sleep(2) # wait for Arduino to configure serial

is_connected = False # Let's create a variable to check if the Arduino is connected

print("Welcome to the RFID reader")
print("Press Ctrl+C to exit")
print("Press 1 to boot the system")
print("Press 2 to list all users")
print("Press 3 to list all tags")
print("Press 4 to assign a tag to a user")
print("Press 5 to add a new user")
print("Press 6 to remove a user")

command = input("Enter command: ")

if command == "1":
    arduino.flushInput() #Let's clean the input buffer before receiving data
    # Send a handshake message to the Arduino board. In the Arduino code, we will use command 1 to start communication
    arduino.write(b'1')
    arduino.flush() #Let's flush the serial port
    raw_data = b'' #Let's create an empty byte string to store the response from the Arduino
    while True:
        # Read the serial port
        data = arduino.readline()
        # Check if the Arduino responded with the handshake message
        if data == b'handshake':
            print("Arduino connected")
            is_connected = True
            break
        else:
            print("Arduino not connected")
            is_connected = False
            break
    # Now, arduino is connected, and we can start reading tags
    while is_connected:
        # Read the serial port
        data = arduino.readline()
        # Check if there is data, if there is no data, we will go back to the beginning of the loop
        if data:
            # Check if the data is a tag ID
            if data.startswith(b'ID:'):
                # Get the tag ID
                tag_id = data[3:].decode('utf-8').strip()
                # Check if the tag has been read before
                if tag_id in tags:
                    # Get the tag from the tags list
                    tag = tags[tag_id]
                    # Get the user from the users list
                    is_user_found = False
                    for user in users:
                        if user['tagId'] == tag_id:
                            # Send a message to the Arduino board with the user name
                            arduino.write(b'Welcome ' + user['name'])
                            arduino.flush() #Let's flush the serial port
                            is_user_found = True
                            break
                    if not is_user_found:
                        # Send a message to the manager! this tag is not assigned to any user
                        print("Unknown tag " + tag_id)
                else:
                    # Add the tag to the tags list
                    tags.append({
                        "tagId": tag_id,
                        "lastRead": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    })
            else:
                print("Error data: " + data)


elif command == "2":
    # List all users
    print(users)
elif command == "3":
    # List all tags
    print(tags)
elif command == "4":
    # Assign a tag to a user, you need to update this code
    print("Assign a tag to a user")
elif command == "5":
    # Add a new user, you need to update this code
    print("Add a new user")
elif command == "6":
    # Remove a user, you need to update this code
    print("Remove a user")
else:
    print("Invalid command")
```

## Arduino code
Now, in the Arduino code, our strategy is going to be the following:
- We will use the Arduino to read the RFID tags and send the data to Python. We will only send the data when a new tag is read.
- When we start the Arduino, we will wait for a handshake message from Python. We will use command 1 to start communication.
- When we receive the handshake command (option 1), we will send a handshake message back to Python.
- We will set a boolean variable to True to indicate that the Arduino is connected.
- We will read the RFID tags and send the data to Python. We will only send the data when a new tag is read.

```c++
// This assumes you have your device working on "local mode" and a switch connected to PIN 15 to turn on/off the remote mode. 
// This example only covers the remote mode.
// Add your libraries and pin definitions here

// This is a boolean variable to check if the Arduino is connected
bool is_connected = false;

void setup() {
  // Add your setup code here
  }

void loop() {
 if (digitalRead(15) == HIGH) {
    // If the switch is on, we will start the remote mode
    // We will wait for a handshake message from Python
    // First, we will wait for a command from Python
    if (Serial.available() > 0) {
      // Read the command if there is data, if not we will go back to the beginning of the main loop
      char option = Serial.read();
      // Check if the command is 1, this is the handshake command
      if (option == '1') {
            // Send a handshake message to Python
            Serial.println("handshake");
            // Set the is_connected variable to true
            is_connected = true;
      }
      if (is_connected) {
        // If the Arduino is connected, we will read the RFID tags
        // Add your code to read the RFID tags here
        // When you read a tag, you will send the tag ID to Python
        // You will send the tag ID using the following format: ID:1234567890
        // You will replace 1234567890 with the tag ID
        // You will use the following code to send the data to Python
        // Serial.print("ID: ");
        // Serial.println(tagId);
        
    }
} 

else {
  // If the switch is off, we will start the local mode
  // Add your local mode code here
}
```

    
